In [1]:
# Library
import numpy as np 
import pandas as pd 

In [110]:
# Read dataset
df_application = pd.read_csv('g_application.tsv',sep='\t')
df_assignee = pd.read_csv('g_assignee_disambiguated.tsv', sep='\t')
df_location = pd.read_csv('g_location_disambiguated.tsv', sep='\t')

C:\Users\PC\AppData\Local\Temp\ipykernel_26668\4144403269.py:2: DtypeWarning: Columns (0,1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_application = pd.read_csv('patent_data/g_application.tsv',sep='\t')


In [53]:
# Save to pickle 
df_application.to_pickle('g_application.pkl')
df_assignee.to_pickle('g_assignee_disambiguated.pkl')
df_location.to_pickle('g_location_disambiguated.pkl')

In [50]:
# Load from pickle
df_application = pd.read_pickle('g_application.pkl')
df_assignee = pd.read_pickle('g_assignee_disambiguated.pkl')
df_location = pd.read_pickle('g_location_disambiguated.pkl')

In [111]:
# Attributes available
i = 0
for df in [df_application, df_assignee, df_location]: 
    print(f"{[i]} : {list(df.columns)}")
    i += 1

[0] : ['application_id', 'patent_id', 'patent_application_type', 'filing_date', 'series_code', 'rule_47_flag']
[1] : ['patent_id', 'assignee_sequence', 'assignee_id', 'disambig_assignee_individual_name_first', 'disambig_assignee_individual_name_last', 'disambig_assignee_organization', 'assignee_type', 'location_id']
[2] : ['location_id', 'disambig_city', 'disambig_state', 'disambig_country', 'latitude', 'longitude', 'county', 'state_fips', 'county_fips']


In [112]:
# Preprocessing
df_application = (
    df_application
        .filter(items=["patent_id", "filing_date", "patent_application_type"])
        .astype({"patent_id" : "str"})
        .assign(filing_date=pd.to_datetime(df_application.filing_date, errors = "coerce"))
        .dropna()
)

df_assignee = (
    df_assignee
        .filter(items=["patent_id", "assignee_id", "disambig_assignee_organization", "location_id"])
        .astype({"patent_id" : "str", "assignee_id" : "str", "location_id" : "str"})
)

df_location = (
    df_location
        .filter(items=["location_id", "disambig_country"])
        .astype({"location_id" : "str", "disambig_country" : "str"})
)   

NOTE: for the `df_assignee` dataset, a `patent_id` can be linked to multiple `assignee_id`. However, the `patent_id` in `df_application` is unique. The `location_id` in `df_location` is also unique.

In [116]:
# Merge 
df = (
    df_assignee
        .merge(df_application, how="left", on="patent_id")
        .merge(df_location, how="left", on="location_id")
)

In [82]:
# Test unique keys 
col = "location_id"
test = df_location
print(len(test))
print(len(np.unique(test[col])))

90730
90730


In [124]:
# Save only after records after 2000
df \
    .query("filing_date >= '2000-01-01'") \
    .to_csv("patents_views.csv")